In [199]:
import numpy as np
import pandas as pd
import rdflib
import json
import xml
import os

import xml.etree.ElementTree as Tree

# Load data

In [18]:
!ls ~/data/waste-disposal-schleswig-holstein/

Q_II_10_j15_SH_korr.xlsx
Q_II_10_j16_SH_korr.xlsx
Q_II_10_j17_SH.xlsx
abfallentsorgung-in-schleswig-holstein-2015-anlagen-der-abfallentsorgung-korrektur.rdf
abfallentsorgung-in-schleswig-holstein-2016-anlagen-der-abfallentsorgung-korrektur.rdf
abfallentsorgung-in-schleswig-holstein-2017-anlagen-der-abfallentsorgung-korrektur.rdf


In [82]:
!head -n10 ~/data/waste-disposal-schleswig-holstein/abfallentsorgung-in-schleswig-holstein-2015-anlagen-der-abfallentsorgung-korrektur.rdf

<?xml version="1.0" encoding="utf-8"?>
<rdf:RDF
  xmlns:foaf="http://xmlns.com/foaf/0.1/"
  xmlns:locn="http://www.w3.org/ns/locn#"
  xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
  xmlns:dcat="http://www.w3.org/ns/dcat#"
  xmlns:dct="http://purl.org/dc/terms/"
  xmlns:skos="http://www.w3.org/2004/02/skos/core#"
  xmlns:vcard="http://www.w3.org/2006/vcard/ns#"
  xmlns:dcatde="http://dcat-ap.de/def/dcatde/"


In [207]:
DATA_PATH = os.path.join(
    os. environ['HOME'],
    'data/waste-disposal-schleswig-holstein/')
YEARS = [2015,2016,2017]
FILENAME = 'abfallentsorgung-in-schleswig-holstein-{}-anlagen-der-abfallentsorgung-korrektur.rdf'

In [210]:
def load_rdf(year):
    XPATH_DATASET = '{http://www.w3.org/ns/dcat#}Dataset[1]' 
    XPATH_SPATIAL = '{http://purl.org/dc/terms/}spatial[1]'
    XPATH_LOCATION = '{http://purl.org/dc/terms/}Location[1]'
    XPATH_GEOMETRY = '{http://www.w3.org/ns/locn#}geometry'
    XPATH_ATTR_DATATYPE = '{http://www.w3.org/1999/02/22-rdf-syntax-ns#}datatype'
    OPENGIS = 'https://www.iana.org/assignments/media-types/application/vnd.geo+json'
    
    filename = FILENAME.format(year)
    tree = Tree.parse(os.path.join(DATA_PATH, filename))
    node = tree.getroot().findall('{}/{}/{}/{}[@{}="{}"]'.format(
        XPATH_DATASET, 
        XPATH_SPATIAL,
        XPATH_LOCATION,
        XPATH_GEOMETRY,
        XPATH_ATTR_DATATYPE,
        OPENGIS))[0]
    jsondata = json.loads(node.text)
    coords = jsondata['coordinates'] # Type: OpenGIS - Multipolygon
    return (year, coords)

annual_coords = map(load_rdf, YEARS)

# Parse Coords into Pandas

In [211]:
next(annual_coords)

(2015,
 [[[[8.5633, 54.7523],
    [8.5859, 54.744],
    [8.5887, 54.741],
    [8.5843, 54.7304],
    [8.597, 54.719],
    [8.5818, 54.7102],
    [8.5719, 54.6926],
    [8.5767, 54.6924],
    [8.5663, 54.6796],
    [8.4849, 54.6803],
    [8.4651, 54.6841],
    [8.4522, 54.6926],
    [8.4249, 54.6952],
    [8.4023, 54.7001],
    [8.3963, 54.7041],
    [8.395, 54.714],
    [8.4044, 54.7301],
    [8.4273, 54.7461],
    [8.4704, 54.7522],
    [8.4856, 54.7495],
    [8.5142, 54.7543],
    [8.5172, 54.751],
    [8.5341, 54.7558],
    [8.5633, 54.7523]]],
  [[[8.705, 54.5575],
    [8.7107, 54.5529],
    [8.7059, 54.5375],
    [8.6967, 54.5238],
    [8.687, 54.521],
    [8.6967, 54.5174],
    [8.6886, 54.5107],
    [8.6872, 54.509],
    [8.6699, 54.4944],
    [8.6381, 54.4894],
    [8.6205, 54.4899],
    [8.5922, 54.5086],
    [8.5873, 54.5223],
    [8.5906, 54.5316],
    [8.5977, 54.5342],
    [8.6261, 54.5355],
    [8.6418, 54.5456],
    [8.6683, 54.5494],
    [8.6863, 54.5571],
    [8.705, 5

In [48]:
dfcoord = pd.concat(map(rdf_to_pandas, annual_coords))